In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
import sys
import pandas as pd
import shutil
import random
import numpy as np

In [2]:
sys.path.insert(0, '..')
from function import *
from createimages import *

In [8]:
artists = pd.read_csv('../raw_data/artists.csv')
artists.head()

,id,name,years,genre,nationality,bio,wikipedia,paintings
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194


In [9]:
#drop id, years, nationality, bio, wikipedia
artists.drop(labels=['id','years','nationality','bio', 'wikipedia'], axis=1, inplace = True)
#rename 'Albrecht Dürer' to 'Albrecht Durer'
artists['name'] = artists['name'].replace(to_replace='Albrecht D.rer', value = 'Albrecht Durer', regex = True)
artists['name'] = artists['name'].str.replace(' ', '_')
#calculate percentage of paintings against total
artists['% of paintings'] = round(artists.paintings/artists.paintings.sum()*100, 2)
artists.head()

,name,genre,paintings,% of paintings
0,Amedeo_Modigliani,Expressionism,193,2.29
1,Vasiliy_Kandinskiy,"Expressionism,Abstractionism",88,1.04
2,Diego_Rivera,"Social Realism,Muralism",70,0.83
3,Claude_Monet,Impressionism,73,0.86
4,Rene_Magritte,"Surrealism,Impressionism",194,2.30


In [10]:
#create data frame with artist name, genre, filename, filepath
path = r'..\raw_images'

#get image information
img_info = get_image_info(path)

#convert lists to dataframe
zipped = zip(['name', 'filename', 'filepath'], img_info)
artlabels = pd.DataFrame(dict(list(zipped)))
artlabels.head()

,name,filename,filepath
0,Albrecht_Durer,Albrecht_Durer_(1).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
1,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
2,Albrecht_Durer,Albrecht_Durer_(100).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
3,Albrecht_Durer,Albrecht_Durer_(101).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
4,Albrecht_Durer,Albrecht_Durer_(102).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...


In [11]:
#extract the artist name and genre
genre = artists.loc[:,['name', 'genre']]

#label each file with their respective genres
artlabels = artlabels.merge(genre, how='left', on='name')
artlabels.head()

,name,filename,filepath,genre
0,Albrecht_Durer,Albrecht_Durer_(1).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
1,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
2,Albrecht_Durer,Albrecht_Durer_(100).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
3,Albrecht_Durer,Albrecht_Durer_(101).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
4,Albrecht_Durer,Albrecht_Durer_(102).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance


In [8]:
#create a new folder for the csv files
datapath = '..\data'
create_new_folder(datapath)

..\data path already exists


In [9]:
#create a new csv file with at most one genre for an artist
#uniqueartlabels = artlabels[~artlabels.genre.str.contains(',')]
#uniqueartlabels.reset_index(drop=True, inplace=True)

newfile = 'artists_clean.csv'
fullpath = os.path.join(datapath, newfile)
artists.to_csv(fullpath)

In [10]:
#create a csv file from the new dataframe
newfile = 'artlabels.csv'
fullpath = os.path.join(datapath, newfile)
artlabels.to_csv(fullpath)

In [60]:
#clear old images to procure a new sample
rawpath = r'..\raw_images'
imagepath = r'..\images'
if os.path.exists(imagepath):
    num_skipped = clear_old_images(imagepath, artists)

Deleted 8446 images


In [61]:
sample_list = get_random_imgs(rawpath, imagepath, artists, 20)

Generated 2002 new images


In [62]:
df_sample = pd.DataFrame(columns=['label', 'filename'])
for i in range(len(sample_list)):
    df_sample.loc[i] = sample_list[i]  

In [63]:
df_sample.shape

(2002, 2)

In [64]:
df_sample.head()

,label,filename
0,Expressionism,Amedeo_Modigliani_133.jpg
1,Expressionism,Amedeo_Modigliani_88.jpg
2,Expressionism,Amedeo_Modigliani_189.jpg
3,Expressionism,Amedeo_Modigliani_83.jpg
4,Expressionism,Amedeo_Modigliani_28.jpg


In [65]:
df_sample.to_csv(r'..\data\sample.csv')